# Ejercicio 2 - Kernel CUDA con stride

En el código proporcionado, implementar el kernel.
---

In [ ]:
# Ejecutar en Google Colab
!pip install numpy matplotlib scikit-image numba cython setuptools

In [ ]:
### EVITAR ERRORES
!uv pip install -q --system numba-cuda==0.4.0

from numba import config
config.CUDA_ENABLE_PYNVJITLINK = 1

In [3]:
import numpy as np
from math import hypot, sqrt
from numba import cuda


# En CPU
def cpu_hypot(a,b):
    return np.hypot(a,b)

# En GPU
@cuda.jit
def gpu_hypot_stride(x, y, out):

    start = cuda.grid(1) # índice global del hilo actual (hilo dentro del bloque como el bloque dentro del grid)
    stride = cuda.gridsize(1) # tamaño total de la grid (nº hilos ejecutándose)

    for i in range(start, x.shape[0], stride): # bucle con stride
        out[i] = abs(x[i])*sqrt(1+(y[i]/x[i])**2) # NO usamos np.hypot (no soportado en CUDA)


n = 1000000 # nº elementos

# CPU vbles
h_a = np.random.uniform(-12, 12, n).astype(np.float32)
h_b = np.random.uniform(-12, 12, n).astype(np.float32)

# GPU vbles
d_a = cuda.to_device(h_a)
d_b = cuda.to_device(h_b)
d_c = cuda.device_array_like(d_b) # output


# Bloques e hilos
blocks = 128
threads_per_block = 64


# Llamada a Kernel
gpu_hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)


# Assert
np.testing.assert_almost_equal(np.hypot(h_a, h_b), d_c.copy_to_host(), decimal=5)


# Medición tiempo

# cpu
print("\nTiempo en CPU:")
%timeit cpu_hypot(h_a, h_b)


#gpu
print("\nTiempo en GPU:")
%timeit gpu_hypot_stride[blocks, threads_per_block](d_a, d_b, d_c)


Tiempo en CPU:
6.27 ms ± 2.01 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)

Tiempo en GPU:
171 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
